## Εργασία στην Επιχειρησιακή Έρευνα

#### Εξάμηνο: Εαρινό 2025
#### Συγγραφέας: Τσαμήτρος Νικόλαος 
#### ΑΕΜ: 10781

### Σχεδιασμός Εβδομαδιαίου Προγράμματος Μαθημάτων

Το ζητούμενο του προβλήματος είναι να καθορίσουμε το εβδομαδιαίο πρόγραμμα δύο τμημάτων που παρακολουθούν τα ίδια μαθήματα, λαμβάνοντας υπόψη τους ακόλουθους χρονικούς περιορισμούς των μαθημάτων και τις απουσίες των καθηγητών. 

- Οι χρονικές ζώνες της κάθε ημέρας είναι 4. Συνεπώς έχουμε **20 διαθέσιμα slots** ανά εβδομάδα (4 ανά ημέρα x 5 ημέρες).
- Τα μαθήματα έχουν διάρκεια 2 ώρες, το κάθε μάθημα καταλαμβάνει δηλαδή 1 slot.
- Οι καθηγητές είναι κοινοί στα δύο τμήματα εκτός από τα μαθήματα της Φυσικής Αγωγής και των Μαθηματικών.
- Περιοροσμοί:
  1. Η πρώτη ζώνη της Δευτέρας (slot 0) δεν είναι διαθέσιμη.
  2. Η Φυσική Αγωγή διδάσκεται μόνο Πέμπτη απόγευμα από τις 14:00 έως τις 16:00 (slot 14).
  3. Ο κ. Λαθοπράξης δεν διδάσκει Δευτέρα πρωί (slot 0 και slot 1).
  4. Η κ. Ινσουλίνα δεν εργάζεται τις Τετάρτες (slots 8, 9, 10, 11)
  5. Το ίδιο μάθημα δεν πρέπει να διδάσκεται δυό φορές την ίδια ημέρα ανά τμήμα.
  6. Το ίδιο μάθημα πρέπει να διδάσκεται ταυτόχρονα σε όλους τους μαθητές του ίδιου τμήματος.

#### Κώδικας

In [1]:
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate 

# Δεδομένα
slots_per_day = 4
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
total_slots = slots_per_day * len(days)
sections = ["S1", "S2"]

# Δομή: (Καθηγητής, Μάθημα, Ώρες στο S1, Ώρες στο S2)
lessons = [
    ("Gesmanidis", "English", 1, 1),
    ("Insoulina", "Biology", 3, 3),
    ("Chartoula", "History-Geography", 2, 2),
    ("Lathopraxis", "Math", 0, 4),
    ("Antiparagwgos", "Math", 4, 0),
    ("Kirκofidou", "Physics", 3, 3),
    ("Platiazwn", "Philosophy", 1, 1),
    ("Mpratsakis", "PE", 1, 0),
    ("Trexalitoula", "PE", 0, 1)
]

# Χαρτογράφηση slot → (μέρα, περίοδος)
slot_day_period = [(day, p) for day in days for p in range(slots_per_day)]

model = Model("School_Timetable")

# Μεταβλητές
x = {}
for teacher, subject, h1, h2 in lessons:
    for sec, h in zip(sections, [h1, h2]):
        if h == 0:
            continue
        for s in range(total_slots):
            x[(teacher, subject, sec, s)] = model.addVar(vtype=GRB.BINARY,
                                                         name=f"x_{teacher}_{subject}_{sec}_{s}")

model.update()

# Περιορισμοί

# 1. Κάθε μάθημα πρέπει να τοποθετηθεί ακριβώς όσες φορές χρειάζεται ανά τμήμα
for teacher, subject, h1, h2 in lessons:
    for sec, h in zip(sections, [h1, h2]):
        if h == 0:
            continue
        model.addConstr(quicksum(x[(teacher, subject, sec, s)] for s in range(total_slots)) == h)

# 2. Ένας καθηγητής δεν μπορεί να διδάσκει σε 2 τμήματα ταυτόχρονα
for teacher, subject, _, _ in lessons:
    for s in range(total_slots):
        expr = quicksum(
            x[(teacher, subj, sec, s)]
            for (t, subj, sec, slot) in x
            if t == teacher and slot == s
        )
        model.addConstr(expr <= 1)

# 3. Κάθε τμήμα μπορεί να έχει μόνο ένα μάθημα ανά slot
for sec in sections:
    for s in range(total_slots):
        expr = quicksum(
            x[(teacher, subject, sec, s)]
            for (teacher, subject, section, slot) in x
            if section == sec and slot == s
        )
        model.addConstr(expr <= 1)

# 4. Περιορισμοί διαθεσιμότητας καθηγητών

# Slot 0 είναι δεσμευμένο για την οργάνωση της μελέτης της εβδομάδας
for key in x:
    if key[3] == 0:
        model.addConstr(x[key] == 0)

# Ινσουλίνα δεν δουλεύει Τετάρτες (slots 8–11)
for s in range(8, 12):
    key1 = ("Insoulina", "Biology", "S1", s)
    key2 = ("Insoulina", "Biology", "S2", s)
    if key1 in x:
        model.addConstr(x[key1] == 0)
    if key2 in x:
        model.addConstr(x[key2] == 0)

# Λαθοπράξης απών Δευτέρα πρωί (slots 0–1)
for s in [0, 1]:
    for sec in sections:
        key = ("Lathopraxis", "Math", sec, s)
        if key in x:
            model.addConstr(x[key] == 0)

# 5. Μαθήματα Φυσικής Αγωγής μόνο Πέμπτη απόγευμα: slot 14 (Thu 14:00–16:00)
allowed_pe_slots = [14]
for (teacher, subject, sec, s) in x:
    if subject == "PE" and s not in allowed_pe_slots:
        model.addConstr(x[(teacher, subject, sec, s)] == 0)

# 6. Το ίδιο μάθημα δεν πρέπει να διδάσκεται δύο φορές την ίδια μέρα ανά τμήμα
for subject in set(subj for _, subj, _, _ in lessons):
    for sec in sections:
        for day_index, day_name in enumerate(days):
            slots_in_day = [
                s for s, (d, _) in enumerate(slot_day_period) if d == day_name
            ]
            model.addConstr(
                quicksum(
                    x[(teacher, subj, sec, s)]
                    for (teacher, subj, s_sec, s) in x
                    if subj == subject and s_sec == sec and s in slots_in_day
                ) <= 1,
                name=f"once_per_day_{subject}_{sec}_{day_name}"
            )

# Αντικειμενική συνάρτηση
model.setObjective(0, GRB.MINIMIZE)

# Επίλυση
model.optimize()

# Εμφάνιση λύσης
print("\nΠρόγραμμα Μαθημάτων:")
for (teacher, subject, sec, s), var in x.items():
    if var.X > 0.5:
        day, period = slot_day_period[s]
        print(f"{day} ({period}): {subject} με {teacher} - {sec}")

for sec in sections:
    table = [["Period/Day"] + days]
    for p in range(slots_per_day):
        row = [f"Period {p+1}"]
        for d in days:
            s = slot_day_period.index((d, p))
            entry = ""
            for (teacher, subject, s_sec, slot), var in x.items():
                if s_sec == sec and slot == s and var.X > 0.5:
                    entry = f"{subject}\n({teacher})"
                    break
            row.append(entry)
        table.append(row)
    
    print(f"\nΠρόγραμμα για Τμήμα {sec}:\n")
    print(tabulate(table, headers="firstrow", tablefmt="grid"))

Set parameter Username
Set parameter LicenseID to value 2668311
Academic license - for non-commercial use only - expires 2026-05-19
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 366 rows, 280 columns and 1182 nonzeros
Model fingerprint: 0xec6875d0
Variable types: 0 continuous, 280 integer (280 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

Πρόγρα

#### Συμπεράσματα 

Τρέχοντας τον κώδικα, προκύπτει το πρόγραμμα του σχολείου που παρουσιάζεται παραπάνω και το οποίο καλύπτει τους περιορισμούς της εκφώνησης.

### Βέλτιστη Χωροθέτηση Αποθηκών

Μια μεγάλη εταιρεία επιθυμεί να εγκαταστήσει νέες αποθήκες προκειμένου να εξυπηρετήσει τη ζήτηση 12 κέντρων πώλησης. Κάθε αποθήκη έχει συγκεκριμένο πάγιο κόστος εγκατάστασης και μέγιστη χωρητικότητα (σε τόνους). Το κόστος μεταφοράς από κάθε αποθήκη προς κάθε κέντρο πώλησης εξαρτάται από την απόσταση και αντιστοιχεί στο συνολικό κόστος για την πλήρη κάλυψη της ζήτησης του εκάστοτε κέντρου.

Ο στόχος είναι να αποφασίσουμε: 
- Ποιες αποθήκες θα λειτουργήσουν
- Πώς θα κατανεμηθούν οι ποσότητες από τις αποθήκες στα κέντρα πώλησης
έτσι ώστε να ελαχιστοποιήσουμε το συνολικό κόστος εγκατάστασης και μεταφοράς διατηρώντας ταυτόχρονα την κάλυψη της ζήτησης του κάθε κέντρου.

Πιο συγκεκριμένα οι περιορισμοί είναι οι ακόλουθοι: 
1. Κάλυψη Ζήτησης: Η ζήτηση κάθε κέντρου πώλησης πρέπει να καλύπτεται πλήρως, είτε από μία είτε από περισσότερες αποθήκες.
2. Χωρητικότητα αποθηκών: Η συνολική ποσότητα που αποστέλλεται από κάθε αποθήκη δεν πρέπει να υπερβαίνει τη μέγιστη χωρητικότητά της.
3. Λειτουργία αποθηκών: Μια αποθήκη μπορεί να αποστείλει προϊόν μόνο αν έχει επιλεγεί να λειτουργήσει.
4. Μη εφικτές μεταφορές: Για ζεύγη αποθήκης–κέντρου πώλησης όπου δεν είναι εφικτή η μεταφορά (το έχω δηλώσει με 0 στον πίνακα κόστους και το αντικαθιστώ με 10^6), η μεταφερόμενη ποσότητα ορίζεται ίση με μηδέν.

#### Κώδικας

In [2]:
from gurobipy import *

transport_costs = [
    [100, 80, 50, 50, 60, 100, 120, 90, 60, 70, 65, 110],
    [120, 90, 60, 70, 65, 110, 140, 110, 80, 80, 75, 130],
    [140, 110, 80, 80, 75, 130, 160, 125, 100, 100, 80, 150],
    [160, 125, 100, 100, 80, 150, 190, 150, 130, 0, 0, 0],
    [190, 150, 130, 0, 0, 0, 180, 150, 50, 50, 60, 100],
    [200, 180, 150, 0, 0, 0, 100, 120, 90, 60, 75, 110],
    [120, 90, 60, 70, 65, 110, 140, 110, 80, 80, 75, 130],
    [120, 90, 60, 70, 65, 110, 140, 110, 80, 80, 75, 130],
    [140, 110, 80, 80, 75, 130, 160, 125, 100, 100, 80, 150],
    [160, 125, 100, 100, 80, 150, 190, 150, 130, 0, 0, 0],
    [190, 150, 130, 0, 0, 0, 200, 180, 150, 0, 0, 0],
    [200, 180, 150, 0, 0, 0, 100, 80, 50, 50, 60, 100]
]

fixed_costs = [3500, 9000, 10000, 4000, 3000, 9000, 9000, 3000, 4000, 10000, 9000, 3500]
capacities = [300, 250, 100, 180, 275, 300, 200, 220, 270, 250, 230, 180]
demand = [120, 80, 75, 100, 110, 100, 90, 60, 30, 150, 95, 120]

M = len(fixed_costs)  # Αποθήκες
N = len(demand)       # Κέντρα πώλησης

INF = 1e6

# Υπολογισμός μοναδιαίου κόστους (€/τόνο)
costs = [
    [
        INF if transport_costs[i][j] == 0 else transport_costs[i][j] / demand[j]
        for j in range(N)
    ]
    for i in range(M)
]

# Μοντέλο
model = Model("Optimal_Warehouse_Placement")

# Μεταβλητές
open_warehouse = model.addVars(M, vtype=GRB.BINARY, name="Open")
ship = model.addVars(M, N, vtype=GRB.CONTINUOUS, name="Ship")

# Αντικειμενική συνάρτηση
model.setObjective(
    quicksum(fixed_costs[i] * open_warehouse[i] for i in range(M)) +
    quicksum(costs[i][j] * ship[i, j] for i in range(M) for j in range(N)),
    GRB.MINIMIZE
)

# Περιορισμοί ζήτησης
for j in range(N):
    model.addConstr(quicksum(ship[i, j] for i in range(M)) == demand[j], name=f"Demand_{j}")

# Περιορισμοί χωρητικότητας
for i in range(M):
    model.addConstr(quicksum(ship[i, j] for j in range(N)) <= capacities[i] * open_warehouse[i], name=f"Capacity_{i}")

# Περιορισμοί για μη εφικτές μεταφορές
for i in range(M):
    for j in range(N):
        if costs[i][j] >= INF:
            model.addConstr(ship[i, j] == 0, name=f"Invalid_{i}_{j}")

# Επίλυση
model.optimize()

# Αποτελέσματα
if model.status == GRB.OPTIMAL:
    print(f"\nΣυνολικό Κόστος: {model.ObjVal:.2f} Ευρώ")
    for i in range(M):
        if open_warehouse[i].X > 0.5:
            print(f"\nΑποθήκη {i+1} ανοίγει:")
            for j in range(N):
                if ship[i, j].X > 0:
                    print(f"  Στέλνει {ship[i, j].X:.1f} τόνους στο Κέντρο Πώλησης {j+1}")
else:
    print("Το πρόβλημα δεν βρέθηκε εφικτό ή δεν είναι βέλτιστο.")


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 45 rows, 156 columns and 321 nonzeros
Model fingerprint: 0x058a571a
Variable types: 144 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 2e+02]
Found heuristic solution: objective 49725.000000
Presolve removed 21 rows and 21 columns
Presolve time: 0.00s
Presolved: 24 rows, 135 columns, 258 nonzeros
Variable types: 123 continuous, 12 integer (12 binary)

Root relaxation: objective 1.574011e+04, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

    

#### Συμπεράσματα

Από τα παραπάνω, παρατηρούμε πως η βέλτιστη λύση που διασφαλίζει την πλήρη κάλυψη της ζήτησης όλων των κέντρων πώλησης έχει συνολικό κόστος 17929.2 Ευρώ. Για την εφαρμογή αυτής πρέπει να ανοίξουν οι αποθήκες 1, 5, 8, 9 και 12. 